In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import xarray as xr
import copy
import os
import sys

import metrics
sys.path.append(os.path.realpath('../split-data/'))
import return_period_tools as tools

import random

import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [2]:
xr.__version__

'0.16.1'

In [3]:
with open('./model_output_for_analysis/nwm_chrt_v2_1d_local.p', 'rb') as fb:
    nwm_results = pkl.load(fb)

with open('./model_output_for_analysis/lstm_time_split1.p', 'rb') as fb:
    lstm_results_time_split1 = pkl.load(fb)
with open('./model_output_for_analysis/mclstm_time_split1.p', 'rb') as fb:
    mclstm_results_time_split1 = pkl.load(fb)
with open('./model_output_for_analysis/sacsma_time_split1.p', 'rb') as fb:
    sacsma_results_time_split1 = pkl.load(fb)

    
with open('./model_output_for_analysis/lstm_time_split2.p', 'rb') as fb:
    lstm_results_time_split2 = pkl.load(fb)
with open('./model_output_for_analysis/mclstm_time_split2.p', 'rb') as fb:
    mclstm_results_time_split2 = pkl.load(fb)
with open('./model_output_for_analysis/sacsma_time_split2.p', 'rb') as fb:
    sacsma_results_time_split2 = pkl.load(fb)
    
with open('./model_output_for_analysis/lstm_return_period_split.p', 'rb') as fb:
    lstm_results_return_period_split = pkl.load(fb)
with open('./model_output_for_analysis/mclstm_return_period_split.p', 'rb') as fb:
    mclstm_results_return_period_split = pkl.load(fb)
with open('./model_output_for_analysis/sacsma_return_period_split.p', 'rb') as fb:
    sacsma_results_return_period_split = pkl.load(fb)

train_split_type_model_set = {'time_split1':{'nwm':nwm_results, 
                                           'lstm':lstm_results_time_split1,
                                            'mc':mclstm_results_time_split1,
                                            'sac':sacsma_results_time_split1},
                              'time_split2':{'nwm':nwm_results, 
                                           'lstm':lstm_results_time_split2,
                                            'mc':mclstm_results_time_split2,
                                            'sac':sacsma_results_time_split2},
                              'return_period_split':{'nwm':nwm_results, 
                                           'lstm':lstm_results_return_period_split,
                                            'mc':mclstm_results_return_period_split,
                                            'sac':sacsma_results_return_period_split}}

range_for_analysis = {'time_split1': [1989,1999],'time_split2': [1996, 2014],'return_period_split':[1996, 2014]}

In [4]:
# Convert flow to   CFS mm -> ft     km^2 -> ft^2    hr->s
conversion_factor = 0.00328084 * 10763910.41671 / 3600 / 24

In [5]:
# Get a list of all the basins in the analysis
#basin_list = list(lstm_results_time_split.keys())

# Camels attributes with RI information
dataName = '../data/camels_attributes.csv'
# load the data with pandas
pd_attributes = pd.read_csv(dataName, sep=',', index_col='gauge_id')

# Add the basin ID as a 8 element string with a leading zero if neccessary
basin_id_str = []
for a in pd_attributes.index.values:
    basin_id_str.append(str(a).zfill(8))
pd_attributes['basin_id_str'] = basin_id_str

In [6]:
#-------------------------------------------------------------------------------------------------
# Solve this problem. I think it is the xarray structures...
# isibleDeprecationWarning: Creating an ndarray from ragged nested sequences 
# (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. 
# If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


In [7]:
def calculate_all_metrics_for_frequency_analysis(analysis_dict, flows, recurrance_interval):

    sims = list(flows.keys())[:-1]

    for metric in metrics.get_available_metrics():

        score = {sim:0 for sim in sims}
    
        analysis_dict[metric]['ri'].append(recurrance_interval)
    
        if metric == 'NSE':
            for sim in sims:
                score[sim] = metrics.nse(flows['obs'],flows[sim])
        if metric == 'MSE':
            for sim in sims:
                score[sim] = metrics.mse(flows['obs'],flows[sim])
        if metric == 'RMSE':
            for sim in sims:
                 score[sim] = metrics.rmse(flows['obs'],flows[sim])
        if metric == 'KGE':
            for sim in sims:
                score[sim] = metrics.kge(flows['obs'],flows[sim])
        if metric == 'Alpha-NSE':
            for sim in sims:
                score[sim] = metrics.alpha_nse(flows['obs'],flows[sim])
        if metric == 'Beta-NSE':
            for sim in sims:
                score[sim] = metrics.beta_nse(flows['obs'],flows[sim])
        if metric == 'Pearson-r':
            for sim in sims:
                score[sim] = metrics.pearsonr(flows['obs'],flows[sim])
        if metric == 'Peak-Timing':
            for sim in sims:
                score[sim] = np.abs(metrics.mean_peak_timing(flows['obs'],flows[sim]))
        if metric == 'FHV':
            for sim in sims:
                score[sim] = metrics.fdc_fhv(flows['obs'],flows[sim])
        if metric == 'FLV':
            for sim in sims:
                score[sim] = metrics.fdc_flv(flows['obs'],flows[sim])
        if metric == 'FMS':
            for sim in sims:
                score[sim] = metrics.fdc_fms(flows['obs'],flows[sim])

        for sim in sims:
            analysis_dict[metric][sim].append(score[sim])

    return

In [14]:
bad_sac=0

#-------------------------------------------------------------------------------------------------
analysis_dict_names = {'time_split1':'frequency_analysis_dict_time_split1.pkl',
                       'time_split2':'frequency_analysis_dict_time_split2.pkl', 
                      'return_period_split':'frequency_analysis_dict_return_period_split.pkl'}
peak_flows_dict_names = {'time_split1':'peak_annual_flows_dict_time_split1.pkl',
                         'time_split2':'peak_annual_flows_dict_time_split2.pkl',
                         'return_period_split':'peak_annual_flows_dict_return_period_split.pkl'}
#-------------------------------------------------------------------------------------------------


for train_split_type in ['time_split1', 'time_split2', 'return_period_split']:
    print('Analyzing ',train_split_type)
    
    #-------------------------------------------------------------------------------------------------
    # Set up lists
    if train_split_type == 'time_split1':
        models = ['lstm', 'mc', 'sac']
        flows = ['lstm', 'mc', 'sac', 'obs']
        models_obs_ri = ['lstm', 'mc', 'sac', 'obs', 'ri']
        models_ri = ['lstm', 'mc', 'sac', 'ri']
    else:
        models = ['nwm', 'lstm', 'mc', 'sac']
        flows = ['nwm', 'lstm', 'mc', 'sac', 'obs']
        models_obs_ri = ['nwm', 'lstm', 'mc', 'sac', 'obs', 'ri']
        models_ri = ['nwm', 'lstm', 'mc', 'sac', 'ri']
    #-------------------------------------------------------------------------------------------------
    
    #-------------------------------------------------------------------------------------------------
    # Place the data here
    analysis_dict_all = {}
    peak_flows_dict = {i:[] for i in models_obs_ri}
    #-------------------------------------------------------------------------------------------------
    
    
    #-------------------------------------------------------------------------------------------------
    #-----LOOP THROUGH BASINS------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------

    for ib, basin_0str in enumerate(train_split_type_model_set[train_split_type]['sac']): 
        basin_int = int(basin_0str)
        print('basin', basin_0str)


        if train_split_type_model_set[train_split_type]['sac'][basin_0str].sum() <=0:
            print('Sacsma run is bad')
            continue
        
        
        #-------------------------------------------------------------------------------------------------
        # Get the NWM data for this basin in an xarray dataset.
        xr_nwm = xr.DataArray(train_split_type_model_set[train_split_type]['nwm'][basin_0str]['streamflow'].values, 
                 coords=[nwm_results[basin_0str]['streamflow'].index], 
                 dims=['datetime'])
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Setting up the dictionary for the single basin results. Then will add to the overall dict.
        analysis_dict = {metric:{model:[] for model in models_ri} for metric in metrics.get_available_metrics()}
        extra_metrics = ['beta-abs', 'peakQ', 'peakRI', 'peakT', 'peakQ-perc', 'peakRI-perc', 'peakT-abs']
        for extra_metric in extra_metrics:
            analysis_dict[extra_metric] = {model:[] for model in models_ri}
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # We need the basin area to convert to CFS, to interpolate the RI from LPIII
        basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
        basin_str = tools.gauge_id_str(basin_int)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Get the log pearson III results
        b17 = tools.read_b17(basin_str)
        #-------------------------------------------------------------------------------------------------


        #-------------------------------------------------------------------------------------------------
        # Get the peak flows, but then cut them to just the validation year.
        peakflows = tools.read_peak_flows(basin_str)
        peakflows['wateryear'] = [int(tools.get_water_year(int(peakflows.iloc[i,0].split('-')[0]), 
                                  int(peakflows.iloc[i,0].split('-')[1]))) for i in range(peakflows.shape[0])]
        peakflows = pd.DataFrame(peakflows.set_index('wateryear'))
        #-------------------------------------------------------------------------------------------------

        #-------------------------------------------------------------------------------------------------
        #----  LOOP THROUGH THE WATER YEARS   ------------------------------------------------------------
        #-------------------------------------------------------------------------------------------------
        for water_year in range(range_for_analysis[train_split_type][0], 
                                range_for_analysis[train_split_type][1]):
            date_from = str(water_year-1)+'-10'
            date_to = str(water_year)+'-09'

            #-------------------------------------------------------------------------------------------------
            # Figure out what the actual recurrence interval is for the basin-year. 
            # We'll use this to categorize the basin-year, but then calc the metrics with the observations.
            if water_year not in list(peakflows.index.values):
                print("water year not in record")
                continue
            peak_date = peakflows.loc[water_year, 0]
            if isinstance(peakflows.loc[water_year, 1], str):
                peak_flow = float(peakflows.loc[water_year, 1].replace(" ", ""))
            else:
                peak_flow = peakflows.loc[water_year, 1]
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Make dictionary with all the flows
            flow_mm = {}
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------        
             # NWM data
            if train_split_type != 'time_split1':
                sim_nwm = xr_nwm.loc[date_from:date_to]
                # convert from CFS to mm/day
                # fm3/s * 3600 sec/hour * 24 hour/day / (m2 * mm/m)
                flow_mm['nwm'] = sim_nwm*3600*24/(basin_area*1000)
            #-------------------------------------------------------------------------------------------------
            # Standard LSTM 
            xrr = train_split_type_model_set[train_split_type]['lstm'][basin_0str]['1D']['xr']['QObs(mm/d)_sim']
            flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------
            # Mass-conserving LSTM data trained on all years
            xrr = train_split_type_model_set[train_split_type]['mc'][basin_0str]['1D']['xr']['QObs(mm/d)_sim']
            flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------        
            # SACSMA 
            df = train_split_type_model_set[train_split_type]['sac'][basin_0str]
            flow_mm['sac'] = df.loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------
            # OBSERVATIONS
            xrr = train_split_type_model_set[train_split_type]['mc'][basin_0str]['1D']['xr']['QObs(mm/d)_obs']
            flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Get the max flow and the location of the max flow. Make them all of same type (np.array).
            max_flow_mm = {flow:np.array(flow_mm[flow].max()) for flow in flows}
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Skip basin years that do not have data
            if np.isnan(max_flow_mm['obs']):
                print('no data, skipping')
                continue
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Get the location of the max flow within the year. If cannot get this value, then need to skip yr
            try:
                max_loc = flow_mm['obs'].values.argmax()
            except:
                print('cannot find max loc')
                continue
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Set the event window around the peak flow
            event_window_days = 10
            max_event_start = pd.Timestamp(date_from) + pd.Timedelta(max_loc - event_window_days, 'day')
            max_event_end = pd.Timestamp(date_from) + pd.Timedelta(max_loc + event_window_days, 'day')
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Get the peak flows within the event window
            max_flow_cfs={}
            for iflow in flows:
                max_flow_cfs[iflow] = np.min([np.max([max_flow_mm[iflow] * \
                                                      basin_area * conversion_factor, 0]),b17.max()])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Calculate the recurrence interval of the maximum annual flow from the log pearson III
            ri={}
            ri['ri'] = tools.interpolate_ri(peak_flow, b17)
            for iflow in flows:
                ri[iflow] = tools.interpolate_ri(max_flow_cfs[iflow], b17)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # The recurrence interval will be None if the flow is too low.
            # We can skip over flows that are too low, because we have plenty of those in our sample
            bad_ri_value_continue = False
            for iflow in ri.keys():
                if not ri[iflow]:
                    print('RI is less than 1')
                    bad_ri_value_continue = True
                    break
            if bad_ri_value_continue:
                continue
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Get all the peak flow values into one place.
            for iflow in flows:
                peak_flows_dict[iflow].append(np.max([max_flow_mm[iflow]]))
            peak_flows_dict['ri'].append(ri['obs'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Make all xarray data similar
            for iflow in flows:
                if iflow == 'nwm': #already in the correct format
                    continue
                if iflow == 'sac': 
                    flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                                   coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
                else:
                    flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                                   coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            if flow_mm['mc'].sum() > 0 and flow_mm['obs'].sum() > 0:
                # This function loops through the metrics and calculates them.
                calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm, ri['ri'])
            else:
                continue
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Calculate the 'simple' flow and timing error. Just peak differences.
            for extra_metric in extra_metrics:
                analysis_dict[extra_metric]['ri'].append(ri['ri'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            for iflow in models:
                max_obs_loc = flow_mm['obs'].argmax().values
                sim_max_window = flow_mm[iflow][np.max([0,max_obs_loc-10]):np.min([365,max_obs_loc+10])]
                max_sim_loc = max_obs_loc + (sim_max_window.argmax().values - 10)
                analysis_dict['peakT'][iflow].append(max_sim_loc - max_obs_loc)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            for iflow in models:
                analysis_dict['peakRI'][iflow].append(ri[iflow] - ri['obs'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            for iflow in models:
                analysis_dict['peakRI-perc'][iflow].append(np.abs(ri[iflow] - ri['obs'])/ri['obs'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            for iflow in models:
                analysis_dict['peakQ'][iflow].append((flow_mm[iflow].max().values - flow_mm['obs'].max().values) * \
                                                         basin_area * conversion_factor)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            for iflow in models:
                analysis_dict['peakQ-perc'][iflow].append(np.abs(flow_mm[iflow].max().values - \
                                                                 flow_mm['obs'].max().values) / \
                                                                 flow_mm['obs'].max().values)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            for iflow in models:
                max_obs_loc = flow_mm['obs'].argmax().values
                sim_max_window = flow_mm[iflow][np.max([0,max_obs_loc-10]):np.min([365,max_obs_loc+10])]
                max_sim_loc = max_obs_loc + (sim_max_window.argmax().values - 10)
                analysis_dict['peakT-abs'][iflow].append(np.abs(max_sim_loc - max_obs_loc))
            #-------------------------------------------------------------------------------------------------


            #------------------------------------------------------------------------------------------------- 
            for iflow in models:
                analysis_dict['beta-abs'][iflow].append(np.abs(analysis_dict['Beta-NSE'][iflow][-1]))
            #-------------------------------------------------------------------------------------------------        

            #------------------------------------------------------------------------------------------------- 
            if train_split_type == 'hp' and ri['ri'] > 50:
                print('recurrance_interval:', ri['ri'])
                for iflow in flows:
                    plt.plot(flow_mm[iflow], label=iflow)
                plt.title('{}, {}'.format(basin_0str, water_year))
                plt.legend()
                plt.show()
                plt.close()
            #------------------------------------------------------------------------------------------------- 

        #-------------------------------------------------------------------------------------------------
        #Now that the basin has been analyzed successfully, add it to the larger dictionary
        analysis_dict_all[basin_0str] = analysis_dict
        #------------------------------------------------------------------------------------------------- 

    #-------------------------------------------------------------------------------------------------
    with open(analysis_dict_names[train_split_type], 'wb') as fb:
        pkl.dump(analysis_dict_all, fb)
    with open(peak_flows_dict_names[train_split_type], 'wb') as fb:
        pkl.dump(peak_flows_dict, fb)

Analyzing  time_split1
basin 01466500
Sacsma run is bad
basin 01022500
basin 01139800
RI is less than 1
basin 01440000
basin 01451800
basin 01195100
basin 01415000
basin 01350000
basin 01435000
basin 01047000
basin 01439500
basin 01414500
basin 01031500
basin 01144000
RI is less than 1
RI is less than 1
basin 01484100
RI is less than 1
RI is less than 1
RI is less than 1
basin 01169000
basin 01413500
basin 01434025
RI is less than 1
RI is less than 1
basin 01057000
basin 01134500
basin 01350140
basin 01333000
basin 01181000
basin 01170100
RI is less than 1
RI is less than 1
RI is less than 1
basin 01142500
RI is less than 1
basin 01423000
basin 01162500
basin 01123000
basin 01365000
basin 01411300
RI is less than 1
basin 01055000
basin 01440400
basin 01139000
basin 01078000
RI is less than 1
basin 01137500
RI is less than 1
basin 01052500
basin 01350080
basin 01073000
basin 01187300
basin 01054200
basin 01491000
basin 01487000
basin 01516500
RI is less than 1
RI is less than 1
RI is le

basin 07335700
RI is less than 1
basin 07359610
RI is less than 1
basin 07362100
basin 07362587
water year not in record
RI is less than 1
RI is less than 1
RI is less than 1
basin 07375000
basin 08013000
basin 08014500
basin 08023080
basin 08050800
basin 08066200
Sacsma run is bad
basin 08066300
basin 08070000
basin 08082700
basin 08070200
basin 08086212
basin 08086290
basin 08101000
basin 08104900
basin 08103900
basin 08109700
basin 08150800
basin 08158700
basin 08164300
basin 08158810
basin 08165300
basin 08164600
basin 08171300
basin 08175000
Sacsma run is bad
basin 08176900
basin 08178880
basin 08189500
basin 08190500
basin 08190000
basin 08195000
basin 08194200
basin 08202700
Sacsma run is bad
basin 08198500
basin 08196000
basin 08200000
basin 08271000
basin 08269000
basin 08267500
basin 08324000
RI is less than 1
basin 08377900
basin 08378500
basin 08380500
basin 09035800
basin 09047700
basin 09035900
basin 09066000
basin 09065500
basin 09066300
basin 09066200
basin 09081600
bas

basin 02298123
basin 02298608
RI is less than 1
basin 02299950
water year not in record
basin 02342933
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
water year not in record
water year not in record
basin 02349900
basin 02361000
RI is less than 1
RI is less than 1
basin 02300700
basin 02350900
basin 02363000
basin 02372250
basin 02369800
basin 02374500
RI is less than 1
basin 02371500
basin 02384540
basin 02381600
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
basin 02395120
RI is less than 1
basin 02415000
basin 02427250
basin 02430085
basin 02464000
basin 02450250
RI is less than 1
RI is less than 1
basin 02464360
basin 02465493
basin 02472000
basin 02479155
RI is less than 1
basin 02469800
RI is less than 1
basin 02472500
basin 02479300
RI is less than 1
w

RI is less than 1
RI is less than 1
basin 06918460
basin 06919500
RI is less than 1
basin 06921070
RI is less than 1
basin 06921200
RI is less than 1
RI is less than 1
RI is less than 1
basin 07057500
RI is less than 1
basin 07060710
Sacsma run is bad
basin 07145700
RI is less than 1
basin 07066000
basin 07142300
basin 07180500
basin 07083000
RI is less than 1
basin 07167500
basin 07184000
basin 07195800
basin 07196900
basin 07197000
basin 07208500
RI is less than 1
basin 07261000
RI is less than 1
basin 07263295
RI is less than 1
basin 07291000
RI is less than 1
basin 07315700
RI is less than 1
basin 07315200
basin 07301410
basin 07299670
basin 07340300
basin 07335700
RI is less than 1
basin 07346045
basin 07359610
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
basin 07362100
RI is less than 1
basin 07362587
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less

basin 01591400
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
basin 01539000
basin 01568000
basin 01580000
RI is less than 1
RI is less than 1
RI is less than 1
basin 01594950
RI is less than 1
basin 01632000
basin 01644000
RI is less than 1
basin 01583500
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
basin 01605500
basin 01632900
basin 01606500
basin 01639500
RI is less than 1
RI is less than 1
RI is less than 1
basin 01596500
basin 01638480
basin 01634500
basin 01664000
RI is less than 1
basin 01666500
basin 01667500
basin 01669520
basin 01669000
basin 02011400
RI is less than 1
RI is less than 1
RI is less than 1
basin 02013000
basin 02014000
basin 02015700
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less

RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
basin 04213000
RI is less than 1
basin 04224775
basin 04216418
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
RI is less than 1
basin 04233000
water year not in record
basin 04221000
RI is less than 1
RI is less than 1
basin 05120500
basin 05057200
basin 05393500
water year not in record
basin 04256000
RI is less than 1
RI is less than 1
basin 05362000
RI is less than 1
basin 04296000
basin 05408000
RI is less than 1
RI is less than 1
RI is less than 1
basin 05291000
basin 05399500
basin 05413500
water year not in record
basin 05414000
basin 05444000
basin 05454000
basin 05458000
basin 05487980
RI is less than 1
RI is less than 1
basin 05466500
basin 05488200
RI is less than 1
basin 05495000
basin 05495500
basin 05489000
RI is less

basin 12167000
RI is less than 1
basin 12175500
RI is less than 1
RI is less than 1
basin 12178100
basin 12186000
basin 12189500
basin 12381400
basin 12375900
basin 12377150
RI is less than 1
RI is less than 1
RI is less than 1
basin 12383500
water year not in record
water year not in record
water year not in record
basin 12390700
basin 12411000
RI is less than 1
basin 12447390
RI is less than 1
RI is less than 1
RI is less than 1
basin 12451000
basin 12488500
basin 13011500
basin 13018300
basin 13011900
RI is less than 1
basin 13023000
basin 13161500
basin 13235000
basin 13240000
water year not in record
basin 13313000
RI is less than 1
basin 14020000
RI is less than 1
basin 13331500
basin 14096850
Sacsma run is bad
basin 14137000
basin 14138800
RI is less than 1
basin 14138870
basin 14138900
RI is less than 1
basin 14139800
RI is less than 1
basin 14141500
RI is less than 1
basin 14154500
basin 14158790
RI is less than 1
basin 14166500
basin 14185000
RI is less than 1
basin 14182500
